In [ ]:
import sys
import inspect
from IPython.display import Markdown

# Add the parent directory to sys.path so 'src' can be imported
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Contextual fraction of 2-qudit states

This python project is for computing the contextual fraction of 2-qudit states with respect to Heisenberg Weyl operators. \


## Heisenberg-Weyl operators
For odd prime dimension $d$, the single qudit Pauli operators are defined by

\begin{equation}
 X \ket{j} = \ket{j + 1} \quad \text{and} \quad Z \ket{j} = \omega^j \ket{j}
\end{equation}

where $\omega = e^{2\pi i/d}$ and the addition is modulo $d$. The Heisenberg-Weyl group is generated by the Pauli operators $X$ and $Z$. An element of the 2-qudit Heisenberg-Weyl group is specified by a symplectic vector $v=[p_1,q_1,p_2,q_2] \in \mathbb{Z}_3^4$: 

\begin{equation}
 W(v) = W([p_1,q_1,p_2,q_2]) = \omega^{2(p_1q_q+p_2q_2)}X^{p_1} Z^{q_1} \otimes X^{p_2}Z^{q_2}
\end{equation}

where we have chosen the phase factor such that

\begin{equation}
 W(v)W(v') = W(v + v'). 
\end{equation}

For a given vector $v \in \mathbb{Z}_3^4$, we can generate the corresponding Heisenberg-Weyl operator using

In [ ]:
import src.utils.operators as operators
Markdown(f"```python\n{inspect.getsource(operators)}\n```")

## Qudit contexts
A context is a set of pairwise commuting observables. We want to identify the maximal contexts of 2-qudit Heisenberg-Weyl operators. 
The operators $W(v)$ and $W(v')$ commute if the vectors $v$ and $v'$ are orthogonal with respect to the symplectic inner product:
\begin{equation}
  [v, v'] = \sum_{i=1}^n (p_i q'_i - p'_i q_i) = 0. 
\end{equation}